In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns  #A statistical plotting library
from sklearn.cluster import KMeans
from kneed import KneeLocator  #A function that helps in optimization of number of clusters from an error curve
from scipy.stats import multivariate_normal as mvn

In [36]:
def load_file_as_arr(Dir):
    #Returns a 3D array DAT
    dat = [] #the list that needs to be converted to 3D array DAT
    for image in os.listdir(Dir):
        f = open(Dir +'/'+image)
        single_image_dat = []
        for line in f:
            single_image_dat.append([float(x) for x in line.strip().split(' ')])
        dat.append(single_image_dat)
    DAT = np.array(dat)
    l = DAT.shape[0]
    m = DAT.shape[1]
    DAT_Tr = DAT.reshape((l*m),-1)
    return DAT,DAT_Tr,l     #len(DAT) is needed for prior prob. calculation as the number of 
                            #examples are different for each class


#Class coast training images        
Dir_ctr = 'Data/Dataset_2B/coast/train'             
coast,coast_tr,n_c = load_file_as_arr(Dir_ctr) 



#Class forest training images
Dir_ftr = 'Data/Dataset_2B/forest/train' 
forest,forest_tr,n_f = load_file_as_arr(Dir_ftr)


#Class mountain training images
Dir_mtr = 'Data/Dataset_2B/mountain/train'  
mountain,mountain_tr,n_m = load_file_as_arr(Dir_mtr)


#Class opencountry training images
Dir_otr = 'Data/Dataset_2B/opencountry/train'  
opencountry,opencountry_tr, n_o = load_file_as_arr(Dir_otr)

#Class street training images
Dir_str = 'Data/Dataset_2B/street/train'  
street,street_tr, n_s = load_file_as_arr(Dir_str)

Dir_coast_dev = 'Data/Dataset_2B/coast/dev' 


#Prior prob. for respective classes as an 1x5 array
initial_weight = np.array([n_c, n_f, n_m, n_o, n_s])
prior_prob = initial_weight/np.sum(initial_weight)


(251, 36, 23)

In [2]:
#KMeans implementation for initialization and optimization of the number of clusters
#Number of clusters for each class eqauls the number of gaussian componenets to be fitted for that class
def K_Clustering(Class,M):
    #Dictionary of the arguments for scikit.KMeans
    KMeans_args = {
        "init" :"random",
        "n_init" : 10,
        "max_iter" : 300,
        "random_state" : 0,
        }
    #Estimation of the optimum number of clusters using elbow method
    std_error = []
    for cluster in range(1,11):
        kmeans = KMeans(n_clusters = cluster , **KMeans_args)
        kmeans.fit(Class)
        std_error.append(kmeans.inertia_)
    if M==0:
        #detecting the elbow point of the curve of 's_err vs K' using kneed, which gives the optimum number of clusters
        curve = KneeLocator(range(1,11), std_error, curve="convex", direction = "decreasing")
        K_opt = curve.elbow
    else:
        #Using Manually entered value for K_opt
        K_opt = M
    #clustering the class in to K_opt clusters 
    kmeans =  KMeans(n_clusters = K_opt , **KMeans_args)
    kmeans.fit(Class)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    return K_opt,labels,centers


In [3]:
#initialization of the parameters using K-Clusters

def Parameters_old(Class,M):
    #Will return a mean(mu)-(K,d) array;
    N,d = np.shape(Class)
    K,lab,mu = K_Clustering(Class,M)
    #gamma contains initial responsibilty values for an example w.r.t each clusters as columns
    gamma = np.array([ [0]*K for i in range(N)])
    for example in range(N):
        for cluster in range(K):
            if lab[example] == cluster:
                gamma[example][cluster]= 1
    return K,mu,gamma


In [38]:
#Defining the Gaussian Mixture Model as a class

class Gaussian_Mixture_Model:
    #Class - Examples of the class to which the Gaussian Componenets need to be fitted
    #Class - N x d matrix, where N is the number of examples and d is the number of features for each example
    #K - Number of Gaussian Components that needs to be fitted
    
    def __init__(self,Class,K,MU,GAMMA,f): 
        self.Class = Class
        self.K = K   #Attribute for Number of clusters
        self.GAMMA = GAMMA          #Attribute for NxK array of posterior responsibity term 
        self.MU = MU                #Attribute for the mean values. An Kxd array.
        self.SIGMA = None           #Attribute for (K,d,d) array of covariances
        self.W = None               #Attribute for prior probabilty, an array of length K
        #self.max_iter = max_iter   #Attribute for the number of iterations
        self.N = len(self.Class)    #Attribute for number of examples available
        self.d = len(self.Class[0]) #Attribute for the number of features in each example
        self.f = f                  #Attribute that acts as switch between diagonal and full covariance matrix
        self.mean_shift = np.reshape(self.Class, (self.N, 1, self.d) ) - np.reshape(self.MU, (1, self.K, self.d) )
    
    def Prior_Probability(self):
        #A function to estimate the (K,) array of prior prob.
        self.W = np.einsum("ij -> j",self.GAMMA) / self.N  
        
    def Mean(self):
      # A function to calculate mean
      self.MU =  ((self.GAMMA).T) @ (self.Class) / np.reshape((self.W*self.N), (self.K, 1)) 
  
    def Covariance_Matrix_Array(self):
        # A function to calculate covariances of the features of the examples
        
        Nk = np.einsum("ij -> j",self.GAMMA)
        self.mean_shift = np.reshape(self.Class, (self.N, 1, self.d) ) - np.reshape(self.MU, (1, self.K, self.d) )
        sigma = np.einsum("nki,nkj->kij", np.einsum("nk,nki->nki", self.GAMMA, self.mean_shift), 
                                   self.mean_shift) / np.reshape(Nk, (self.K, 1, 1))
            
        if self.f==1: #Case where we use full diagonal covariance matrix
            self.SIGMA = sigma
        
        if self.f==0: #Case where we use a diagonal covariance matrix
            I = np.identity(self.d,dtype=int) #An identity matrix of the size equal to number of feature
            self.SIGMA = np.einsum("kij,ij -> kij",sigma,I)
  

    def Gaussian_Prob(self):
        #This function accounts for our assumption that the conditional distribution of an example is a Gaussian.
        
        self.Covariance_Matrix_Array()           #SIGMA gets updated to the full covariance matrix
        SIGMA_inv = np.linalg.inv(self.SIGMA)     #Inverse of the covariance matrix
        
        norm = np.sqrt(((2*np.pi)**self.d)*np.linalg.det(self.SIGMA))  #Normalisation term of the Gaussian dist.
        #Exponential term of the Gaussian
        expo = np.exp(-0.5*(np.einsum("nkj,nkj->nk", np.einsum("nki,kij->nkj", self.mean_shift, SIGMA_inv),
                                     self.mean_shift)))  
        
        #Prob_mat is an (NxK)-array that contains Gaussian Prob. of the various examples to belong to 
        #respective clusters 
        Prob_mat =  expo / norm
        return Prob_mat
    
    
    def Expectation_Step(self):
        #In this step we update the values of the responsibilty term
        
        N = self.Gaussian_Prob()
        self.W =  np.einsum("ij -> j",self.GAMMA) / self.N  #Prior probability array
        Num =  N * self.W
        Den = np.reshape(np.sum(Num, axis=1), (self.N, 1) )
        self.GAMMA = Num/Den
    
      
    def Maximization_Step(self): 
        #In this step we updtae the various parameters
        
        #Updation of GAMMA
        self.Expectation_Step()
        
        #Updation of W
        self.Prior_Probability()
        
        #Updation of Mean MU
        self.Mean()
        
        #Updation of Covariance Matrix SIGMA
        self.Covariance_Matrix_Array()
      
   
    def Log_Likelihood(self):
      
      llhd = np.sum(np.log(self.Gaussian_Prob() @ self.W))
    
      return llhd
      
    
    def fit(self,max_iter,threshold):
        
        log_likelihoods = []  #Attribute for 1D array that contains Log_Likelihood values. Size depends on the number iterations
                                #required to converge
        
        
        for i in range(max_iter):
            self.Expectation_Step()   #Updates Gamma
            self.Maximization_Step()  #Updates all the other parameters
            log_likelihoods.append(self.Log_Likelihood())
            #An if conditional for the requirement of convergence
            if (i!=0) & ((log_likelihoods[i] - log_likelihoods[i-1]) < threshold):
                    break
                    
        print("Number of iterations to converge:" ,i)
        
        
#         #Plotting log_likelihood vs iterations, comment out if not needed
#         sns.set_style("darkgrid")         #setting the plot style
#         fig = plt.figure(figsize=(10,10))
#         ax0 = fig.add_subplot(111) 
#         ax0.set_title('Log-Likelihood')
#         ax0.plot(range(i+1),log_likelihoods)  
        
#         #Plot of the fitted Gaussians for each class
#         x,y = np.meshgrid(np.sort(self.Class[:,0]),np.sort(self.Class[:,1])) # the meshgrid for the plot
#         XY = np.array([x.flatten(),y.flatten()]).T 
#         sns.set_style("darkgrid")         #setting the plot style
#         fig1 = plt.figure(figsize=(10,10))
#         ax1 = fig1.add_subplot(111)
#         ax1.set_title('Fitted Gaussians')
#         ax1.scatter(self.Class[:,0],self.Class[:,1],c= 'r')
#         for mu,sigma in zip(self.MU,self.SIGMA):
#             multi_normal = mvn(mean=mu,cov=sigma)
#             ax1.contour(np.sort(self.Class[:,0]),np.sort(self.Class[:,1]),multi_normal.pdf(XY).reshape(len(self.Class),len(self.Class)),colors='black',alpha=0.3)
            
    
    def Class_Prob(self,Y):
            #call this for prediction of 1b
            #A function that returns Prob. for a unlabelled vector Y to belong to a class
            #Pred_Prob = []
            Multi_Gauss = []
            for mu,sigma in zip(self.MU,self.SIGMA):
                Multi_Gauss.append(mvn(mean=mu,cov=sigma).pdf(Y)) #An array of Multi-Variate Gaussian Prob of various clusters                                                                         
            Wt_Gauss = np.einsum("i,i->i",self.W,Multi_Gauss) #An array of weighted probabilities
            Pred_Prob =np.sum(Wt_Gauss)  
            return Pred_Prob
        
    def Class_Prob_set_features(self,L):
        #call this for prediction of 2b
        #A function that returns Prob. for a set of feature vectors. For example, the set of all local feature 
        #vectors of an image. L will be an 2D array-(nxd).
        n = L.shape[0]   #Number of local feature row vectors
        d = L.shape[1]   #dimension  of feature space
        Prob_nfeature_list = []
        for n_feature in range(n):
            Prob_nfeature_list.append(self.Class_Prob(L[n_feature,:d]))
        
        Prob_nfeature_arr = np.array(Prob_nfeature_list)
        Pred_Prob = np.product(Prob_nfeature_arr)
        return Pred_Prob
            

In [35]:
#Fitting GMM for class coast
for clusters in range(1,9):
    K,MU,GAMMA = Parameters_old(coast_tr,clusters)     #0 as the second argument chooses by default K_opt estimated using elbow method. 
                                                       #If not pass the number of clusters needed
    gmm_coast = Gaussian_Mixture_Model(coast_tr,K,MU,GAMMA,1) #0 as the last argument -> diagonal covariance matrix
    gmm_coast.fit(max_iter=1000,threshold = 1e-3)
    gmm_coast
    
    
    
    
    


Number of iterations to convegre: 1
Number of iterations to convegre: 75
Number of iterations to convegre: 49
Number of iterations to convegre: 101
Number of iterations to convegre: 68
Number of iterations to convegre: 62
Number of iterations to convegre: 61
Number of iterations to convegre: 83


In [34]:
gmm_coast.MU.shape

(9, 23)